# ExoNAMD API v1.0

## Summary

The observer wants to compute the relative and/or absolute NAMD of:
- a given multiplanetary system;
- a subset of multiplanetary systems;
- all the known ones.

This tool handles all of the above cases.

In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime
from datetime import timedelta

pd.options.display.max_columns = 20
pd.options.display.max_rows = 30
pd.options.mode.copy_on_write = True
import warnings

warnings.filterwarnings("ignore")

from exonamd.utils import ROOT
from exonamd.utils import fetch_aliases
from exonamd.utils import update_host
from exonamd.utils import update_planet

### Task 1: getting the data

This task retrieves the parameters of confirmed systems from the NASA Exoplanet Archive database, and stores them in a local database.

In [38]:
# Define the URL for the API
url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync"

# Define the date you want to filter by
from_scratch = True  # change to False in production
if from_scratch:
    latest = datetime.strptime("1990-01-01", "%Y-%m-%d")  # Example date
else:
    df_old = pd.read_csv(os.path.join(ROOT, "data", "exo.csv"))
    latest = df_old["rowupdate"].max()
    latest = datetime.strptime(latest, "%Y-%m-%d")  # Example date
    latest = latest - timedelta(days=1)

# Convert the date to a string in the format 'YYYY-MM-DD'
latest = latest.strftime("%Y-%m-%d")

# Define the multiplicity you want to filter by
multiplicity = 5

# Define the SQL query to retrieve the required data
query = f"""
SELECT 
    hostname, 
    pl_name, 
    default_flag,
    rowupdate,
    sy_pnum, 
    st_rad,
    st_mass,
    pl_orbper,
    pl_orbsmax, 
    pl_orbsmaxerr1, 
    pl_orbsmaxerr2, 
    pl_rade,
    pl_radeerr1,
    pl_radeerr2,
    pl_bmasse, 
    pl_bmasseerr1, 
    pl_bmasseerr2, 
    pl_orbeccen, 
    pl_orbeccenerr1, 
    pl_orbeccenerr2, 
    pl_orbincl, 
    pl_orbinclerr1, 
    pl_orbinclerr2,
    pl_trueobliq,
    pl_trueobliqerr1,
    pl_trueobliqerr2,
    pl_ratdor,
    pl_ratror
FROM ps
WHERE
    sy_pnum > '{multiplicity}'
    AND rowupdate > '{latest}'
"""

# Define the parameters for the request
params = {
    "query": query,
    "format": "json",
}

# Make the request to the API
response = requests.get(url, params=params)

if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    # Convert the JSON data to a pandas DataFrame
    df = pd.DataFrame(data)
else:
    print(f"Error: {response.status_code}")

### Task 2: dealing with the aliases

Fetch aliases

In [39]:
aliases = fetch_aliases(df["hostname"].unique())

Fetched 13/13 entries on try 1


Curate aliases

In [40]:
# # For testing
# df2 = df.copy()
# df2.loc[df2['hostname'] == 'Kepler-20', 'hostname'] = 'KOI-70'
# df2.loc[df2['hostname'] == 'Kepler-80', 'hostname'] = 'KIC 4852528'
# df2['hostname'] = df2.apply(update_host, args=(aliases, True), axis=1)
df["hostname"] = df.apply(update_host, args=(aliases, False), axis=1)

# # For testing
# df2 = df.copy()
# df2.loc[df2['pl_name'] == 'Kepler-20 c', 'pl_name'] = 'KOI-70.01'
# df2.loc[df2['pl_name'] == 'Kepler-11 b', 'pl_name'] = 'KOI-157 b'
# df2['pl_name'] = df2.apply(update_planet, args=(aliases, True), axis=1)
df["pl_name"] = df.apply(update_planet, args=(aliases, False), axis=1)

In [41]:
# Double check that the names are consistent

for hostname in df["hostname"].unique():
    df_host = df[df["hostname"] == hostname]
    names = df_host["pl_name"]
    if len(set([name[:3] for name in names])) > 1:
        print(f"Inconsistent name for {hostname}")

In [42]:
# df.to_csv(os.path.join(ROOT, "data", "task2.csv"), index=False)

### Task 3: computing missing values (if any) from simple equations

In [43]:
from exonamd.utils import solve_a_rs
from exonamd.utils import solve_rprs
from exonamd.utils import solve_a_period


def complete_values(row):

    sma = row["pl_orbsmax"]
    ars = row["pl_ratdor"]
    rstar = row["st_rad"]
    rplanet = row["pl_rade"]
    rprs = row["pl_ratror"]
    period = row["pl_orbper"]
    mstar = row["st_mass"]

    # Rank groups
    a_rs_ = np.isnan(sma) + np.isnan(ars) + np.isnan(rstar)
    rprs_ = np.isnan(rplanet) + np.isnan(rprs) + np.isnan(rstar)
    a_period_ = np.isnan(period) + np.isnan(sma) + np.isnan(mstar)
    solve_order = np.argsort([a_rs_, rprs_, a_period_])
    for i in solve_order:
        if i == 0:
            # Solve semi-major axis -- stellar radius system of equations.
            solution = solve_a_rs(sma, rstar, ars)
            sma, rstar, ars = solution
        elif i == 1:
            # Solve planet radius -- stellar radius system of equations.
            solution = solve_rprs(rplanet, rstar, rprs)
            rplanet, rstar, rprs = solution
        elif i == 2:
            # Solve period-sma-mstar system of equations.
            solution = solve_a_period(period, sma, mstar)
            period, sma, mstar = solution

    return sma, ars, rstar, rplanet, rprs, period, mstar

In [44]:
df[
    [
        "pl_orbsmax",
        "pl_ratdor",
        "st_rad",
        "pl_rade",
        "pl_ratror",
        "pl_orbper",
        "st_mass",
    ]
] = df.apply(complete_values, axis=1, result_type="expand")

### Task 4: storing the curated database

In [45]:
if not from_scratch:
    df_new = df.copy()
    df = pd.concat([df_new, df_old], ignore_index=True)
    df = df.drop_duplicates(keep="last")

df.to_csv(os.path.join(ROOT, "data", "exo.csv"), index=False)

### Task 5: input missing values (if any) by interpolation

Deal with None values

In [ ]:
df = df.replace({None: np.nan, "": np.nan})

Use nanmedian to thin down the data

In [33]:
keep_indices = []

for planet in df["pl_name"].unique():
    df_planet = df[df["pl_name"] == planet]
    # we average the parameters for each planet in the system
    # so there will be for instance 2 rows correpsonding to the same planet
    # and we will have the average of the parameters
    default_index = df_planet[df_planet["default_flag"] == 1].index
    # print(f"Processing {planet}, default indices: {default_index}")
    for col in df_planet.columns[4:]:
        # take the average of the values (ignoring NaNs)
        avg = np.nanmedian(df_planet[col].values)
        # overwrite the values in the original df where default_flag = 1
        # print(f"Overwriting {col} for {planet} with {avg}")
        df.loc[default_index, col] = avg
    keep_indices.extend(default_index)

df.drop(df.index[~df.index.isin(keep_indices)], inplace=True)


# CHECK THAT THIS WORKS

Instantiate flags

In [47]:
df["flag"] = "0"

In [51]:
df

,hostname,pl_name,default_flag,rowupdate,sy_pnum,st_rad,st_mass,pl_orbper,pl_orbsmax,pl_orbsmaxerr1,...,pl_orbeccenerr2,pl_orbincl,pl_orbinclerr1,pl_orbinclerr2,pl_trueobliq,pl_trueobliqerr1,pl_trueobliqerr2,pl_ratdor,pl_ratror,flag
0,Kepler-20,Kepler-20 d,0,2017-05-08,6,0.93,0.88,77.611443,0.341100,NaN,...,NaN,89.940,NaN,NaN,NaN,NaN,NaN,84.36,0.025680,0
1,Kepler-20,Kepler-20 d,0,2024-02-21,6,NaN,0.94,77.611599,0.349066,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,Kepler-20,Kepler-20 d,0,2014-11-21,6,0.93,0.88,77.611599,0.341100,NaN,...,NaN,89.950,NaN,NaN,NaN,NaN,NaN,83.25,0.025920,0
3,Kepler-20,Kepler-20 d,0,2018-09-04,6,0.89,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.025629,0
4,Kepler-20,Kepler-20 d,0,2017-03-28,6,0.94,0.91,77.611840,0.345300,0.0041,...,NaN,89.570,0.043,-0.048,NaN,NaN,NaN,78.30,0.026700,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,HD 110067,HD 110067 c,1,2023-12-05,6,0.79,0.80,13.673694,0.103900,0.0013,...,NaN,89.687,0.163,-0.163,NaN,NaN,NaN,28.37,0.027800,0
452,HD 110067,HD 110067 d,1,2023-12-05,6,0.79,0.80,20.519617,0.136200,0.0017,...,NaN,89.248,0.046,-0.046,NaN,NaN,NaN,37.21,0.033200,0
453,HD 110067,HD 110067 e,1,2023-12-05,6,0.79,0.80,30.793091,0.178500,0.0022,...,NaN,89.867,0.089,-0.089,NaN,NaN,NaN,48.77,0.022600,0
454,HD 110067,HD 110067 f,1,2023-12-05,6,0.79,0.80,41.058540,0.216300,0.0026,...,NaN,89.673,0.046,-0.046,NaN,NaN,NaN,59.08,0.030260,0


In [26]:
# code here

### Task 6: storing the curated+interpolated database

In [27]:
# code here

In [ ]:
df[df["pl_trueobliq"].notnull()][
    ["pl_name", "pl_trueobliq", "pl_trueobliqerr1", "pl_trueobliqerr2"]
]